# Import libraries

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers

In [4]:
folder_path = './Tank1'

file_list = glob.glob(folder_path + '/*.xlsx')

dfs = []

for file in file_list:
    df = pd.read_excel(file, parse_dates=['created_at'])
    dfs.append(df)
    
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.head()

,created_at,entry_id,field1
0,2018-02-14 04:50:55,1,40
1,2018-02-14 04:51:33,2,39
2,2018-02-14 04:52:12,3,40
3,2018-02-14 04:52:51,4,40
4,2018-02-14 04:53:30,5,40


# Convert the date column

In [5]:
merged_df["created_at"] = pd.to_datetime(merged_df["created_at"], format='%d/%m/%Y %H:%M')

merged_df["year"] = merged_df["created_at"].dt.year
merged_df["month"] = merged_df["created_at"].dt.month
merged_df["day"] = merged_df["created_at"].dt.day
merged_df["hour"] = merged_df["created_at"].dt.hour
merged_df["minute"] = merged_df["created_at"].dt.minute
merged_df.head()

,created_at,entry_id,field1,year,month,day,hour,minute
0,2018-02-14 04:50:55,1,40,2018,2,14,4,50
1,2018-02-14 04:51:33,2,39,2018,2,14,4,51
2,2018-02-14 04:52:12,3,40,2018,2,14,4,52
3,2018-02-14 04:52:51,4,40,2018,2,14,4,52
4,2018-02-14 04:53:30,5,40,2018,2,14,4,53


In [6]:
null_data = merged_df.isna().sum()
duplicate_rows = merged_df.duplicated()
print('Nulls', null_data)
print('Duplicates', duplicate_rows)

Nulls created_at    0
entry_id      0
field1        0
year          0
month         0
day           0
hour          0
minute        0
dtype: int64
Duplicates 0        False
1        False
2        False
3        False
4        False
         ...  
89392    False
89393    False
89394    False
89395    False
89396    False
Length: 89397, dtype: bool


# Check for outliers


In [7]:
# Detecting outliers using Z-score or IQR
# Let's focus on the "field1" column for outlier detection
Q1 = merged_df["field1"].quantile(0.25)
Q3 = merged_df["field1"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


# Identify and remove outliers
df_no_outliers = merged_df[
    (merged_df["field1"].iloc[:] >= lower_bound) &
    (merged_df["field1"].iloc[:] <= upper_bound)
]

# train test split

In [8]:
# Splitting data into features (X) and target (y)
X = df_no_outliers.drop(["field1", "created_at"], axis=1)  # Features excluding "field1" and "created_at"
y = df_no_outliers["field1"]  # Target variable
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
# Applying Standard Scaling to the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
from sklearn.metrics import mean_squared_error


# Build a Deep Neural Network (DNN) model using Keras
model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer with 64 neurons and ReLU activation
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
    layers.Dense(1)  # Output layer (single neuron)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Epoch 1/50
1788/1788 [==============================] - 2s 925us/step - loss: 466.6266 - val_loss: 198.1864
Epoch 2/50
1788/1788 [==============================] - 2s 950us/step - loss: 173.4232 - val_loss: 154.0256
Epoch 3/50
1788/1788 [==============================] - 2s 911us/step - loss: 134.8502 - val_loss: 123.0898
Epoch 4/50
1788/1788 [==============================] - 2s 933us/step - loss: 115.8593 - val_loss: 110.4389
Epoch 5/50
1788/1788 [==============================] - 2s 860us/step - loss: 103.7178 - val_loss: 98.0646
Epoch 6/50
1788/1788 [==============================] - 2s 854us/step - loss: 93.0317 - val_loss: 88.8301
Epoch 7/50
1788/1788 [==============================] - 2s 879us/step - loss: 83.7538 - val_loss: 80.5912
Epoch 8/50
1788/1788 [==============================] - 2s 932us/step - loss: 75.1740 - val_loss: 72.6766
Epoch 9/50
1788/1788 [==============================] - 2s 903us/step - loss: 68.5910 - val_loss: 67.5793
Epoch 10/50
1788/1788 [==============

In [11]:
from sklearn.metrics import r2_score

# Build a Deep Neural Network (DNN) model using Keras
model = keras.Sequential([
    layers.Input(shape=(X_train_scaled.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer with 64 neurons and ReLU activation
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 neurons and ReLU activation
    layers.Dense(1)  # Output layer (single neuron)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
y_pred = model.predict(X_test_scaled)

# Calculate R-squared value
r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)

Epoch 1/50
1788/1788 [==============================] - 2s 940us/step - loss: 473.0208 - val_loss: 211.4472
Epoch 2/50
1788/1788 [==============================] - 1s 821us/step - loss: 179.3819 - val_loss: 161.9510
Epoch 3/50
1788/1788 [==============================] - 1s 818us/step - loss: 152.4178 - val_loss: 145.0368
Epoch 4/50
1788/1788 [==============================] - 1s 821us/step - loss: 134.7776 - val_loss: 126.5699
Epoch 5/50
1788/1788 [==============================] - 2s 858us/step - loss: 117.0270 - val_loss: 114.6375
Epoch 6/50
1788/1788 [==============================] - 2s 836us/step - loss: 102.6768 - val_loss: 96.5255
Epoch 7/50
1788/1788 [==============================] - 1s 833us/step - loss: 89.6590 - val_loss: 85.5621
Epoch 8/50
1788/1788 [==============================] - 1s 819us/step - loss: 79.6792 - val_loss: 78.0141
Epoch 9/50
1788/1788 [==============================] - 1s 829us/step - loss: 72.3278 - val_loss: 72.0016
Epoch 10/50
1788/1788 [============

In [13]:
# Reshape data for LSTM and  gru (samples, timesteps, features)
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])


# LSTM MODEL

In [14]:
# LSTM Model
from tensorflow.keras.layers import LSTM, Dense

lstm_model = Sequential([
    LSTM(128, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True),
    LSTM(64, activation='relu'),
    Dense(1)  # Output layer
])

lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Use early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lstm_model.fit(X_train_reshaped, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the LSTM model
y_pred_lstm = lstm_model.predict(X_test_reshaped)

# Calculate and print R-squared
r_squared_lstm = r2_score(y_test, y_pred_lstm)
print("LSTM R-squared:", r_squared_lstm)

Epoch 1/50
1788/1788 [==============================] - 6s 3ms/step - loss: 442.6327 - val_loss: 180.2170
Epoch 2/50
1788/1788 [==============================] - 5s 3ms/step - loss: 163.0885 - val_loss: 143.0230
Epoch 3/50
1788/1788 [==============================] - 5s 3ms/step - loss: 110.0673 - val_loss: 94.1435
Epoch 4/50
1788/1788 [==============================] - 5s 3ms/step - loss: 80.3962 - val_loss: 73.5287
Epoch 5/50
1788/1788 [==============================] - 4s 2ms/step - loss: 66.3897 - val_loss: 61.0096
Epoch 6/50
1788/1788 [==============================] - 5s 3ms/step - loss: 55.2738 - val_loss: 50.5500
Epoch 7/50
1788/1788 [==============================] - 5s 3ms/step - loss: 45.4119 - val_loss: 41.8408
Epoch 8/50
1788/1788 [==============================] - 5s 3ms/step - loss: 39.4259 - val_loss: 36.9045
Epoch 9/50
1788/1788 [==============================] - 5s 3ms/step - loss: 34.7639 - val_loss: 32.4323
Epoch 10/50
1788/1788 [==============================] - 4s

# GRU Model

In [15]:
# GRU Model
from tensorflow.keras.layers import GRU, Dense

gru_model = Sequential([
    GRU(128, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True),
    GRU(64, activation='relu'),
    Dense(1)  # Output layer
])

gru_model.compile(optimizer='adam', loss='mean_squared_error')

# Use early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

gru_model.fit(X_train_reshaped, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the GRU model
y_pred_gru = gru_model.predict(X_test_reshaped)

# Calculate and print R-squared
r_squared_gru = r2_score(y_test, y_pred_gru)
print("GRU R-squared:", r_squared_gru)

Epoch 1/50
1788/1788 [==============================] - 6s 2ms/step - loss: 369.6432 - val_loss: 191.0354
Epoch 2/50
1788/1788 [==============================] - 4s 2ms/step - loss: 180.8256 - val_loss: 167.0408
Epoch 3/50
1788/1788 [==============================] - 4s 2ms/step - loss: 136.5553 - val_loss: 107.8628
Epoch 4/50
1788/1788 [==============================] - 4s 2ms/step - loss: 97.4095 - val_loss: 87.2535
Epoch 5/50
1788/1788 [==============================] - 4s 2ms/step - loss: 75.3096 - val_loss: 62.9872
Epoch 6/50
1788/1788 [==============================] - 4s 2ms/step - loss: 51.8600 - val_loss: 49.2314
Epoch 7/50
1788/1788 [==============================] - 4s 2ms/step - loss: 39.4083 - val_loss: 38.5482
Epoch 8/50
1788/1788 [==============================] - 4s 2ms/step - loss: 33.1845 - val_loss: 30.2704
Epoch 9/50
1788/1788 [==============================] - 4s 2ms/step - loss: 28.6242 - val_loss: 27.2737
Epoch 10/50
1788/1788 [==============================] - 4